## Perturbations

In [24]:
import torch
from perturbations import perturbed

dtype = torch.double
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

kwargs = {"dtype": dtype, "device": device}

DEPTH = 4
DIM = 2**DEPTH - 1

path_inds = torch.zeros(DEPTH, DIM).long()
path_inds[0] = torch.arange(start=0, end=DIM).long()
path_inds[1] = ((path_inds[0]-1)/2).long()
path_inds[2] = ((path_inds[1]-1)/2).long()
path_inds[3] = ((path_inds[2]-1)/2).long()

def best_path(inputs):
    """Returns the ranks of the input values among the given axis."""
    scores = inputs[..., path_inds].sum(dim=-2).squeeze(dim=-2)
    best_scores_ind = scores[..., int(DIM/2):DIM].argmax(dim=-1, keepdim=True) + int(DIM/2)
    best_path = torch.zeros_like(inputs)
    # This handling of best_path_inds doesn't quite work as intended with 3+ dim input
    best_path_inds = path_inds[...,best_scores_ind].squeeze(-1).transpose(-2, -1)
    best_path.scatter_(dim=-1, index=best_path_inds, value=1.0)
    return best_path

# We initialize a random tensor
x = torch.rand([128, DIM]).to(**kwargs)
print(x)

best_path(torch.rand(10, DIM))

tensor([[0.9984, 0.6346, 0.6157,  ..., 0.0390, 0.9799, 0.8945],
        [0.4414, 0.2094, 0.3482,  ..., 0.9080, 0.4936, 0.1060],
        [0.7729, 0.5911, 0.7002,  ..., 0.0216, 0.6762, 0.2952],
        ...,
        [0.7183, 0.4194, 0.7914,  ..., 0.5736, 0.4844, 0.3104],
        [0.0254, 0.8569, 0.8162,  ..., 0.1626, 0.6628, 0.0663],
        [0.5201, 0.3212, 0.9317,  ..., 0.0888, 0.2884, 0.3358]],
       dtype=torch.float64)


tensor([[1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [25]:
# Turn its grad on, since we will change this tensor to minimize our loss
x.requires_grad = True
y_true = torch.Tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

print("Initially, the values in our tensor do not result in the desired argsort")
print(best_path(x))
print(y_true)

torch.norm(perturbed(best_path, device=device)(x) - y_true, dim=-1)

# FenchelYoungLoss(ranks, device=device)(y_true, x)

# Initialize an SGD optimizer and do 200 steps
optim = torch.optim.SGD([x], 1.0)
# optim = torch.optim.LBFGS([x], .01)


Initially, the values in our tensor do not result in the desired argsort
tensor([[1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.]], dtype=torch.float64)
tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])


In [26]:
%%time 
BASE = 10

for iteration in range(10*BASE+1):
    def closure():
        optim.zero_grad()
    #     criterion = FenchelYoungLoss(ranks, sigma=0.25, device=device)
    #     loss = criterion(x, y_true).sum()
        perturbed_x = perturbed(best_path, num_samples=100, sigma=0.25, noise='gumbel', device=device)(x)
        loss = torch.norm(perturbed_x - y_true, dim=-1).mean()
        loss.backward()
        return loss
    optim.step(closure)
    if iteration % BASE== 0:
        perturbed_x = perturbed(best_path, num_samples=100, sigma=0.25, noise='gumbel', device=device)(x)
        loss = torch.norm(perturbed_x - y_true, dim=-1).mean()
#         print(perturbed_x)
        print(torch.abs(best_path(x) - y_true).sum().item())
        print(loss.item())
        
print("SGD has run???")

530.0
1.5222587678283674
396.0
1.3643415413888658
296.0
1.2081967526769732
236.0
1.0552331247352917
172.0
0.9277827876056872
150.0
0.8039533755362521
134.0
0.7034852589045049
110.0
0.6208235629400555
82.0
0.5529975952964732
74.0
0.49107839130080644
68.0
0.4413605019335051
SGD has run???
Wall time: 1.31 s


In [27]:
best_path(x).sum(dim=0)

tensor([128., 128.,   0., 121.,   7.,   0.,   0., 101.,  20.,   2.,   5.,   0.,
          0.,   0.,   0.], dtype=torch.float64)

## Perturbed Fixed Noise

In [15]:
import torch
from perturbations import fixed_noise_perturbed, get_presampled_noises

dtype = torch.double
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

kwargs = {"dtype": dtype, "device": device}

DEPTH = 4
DIM = 2**DEPTH - 1

path_inds = torch.zeros(DEPTH, DIM).long()
path_inds[0] = torch.arange(start=0, end=DIM).long()
path_inds[1] = ((path_inds[0]-1)/2).long()
path_inds[2] = ((path_inds[1]-1)/2).long()
path_inds[3] = ((path_inds[2]-1)/2).long()

def best_path(inputs):
    """Returns the ranks of the input values among the given axis."""
    scores = inputs[..., path_inds].sum(dim=-2).squeeze(dim=-2)
    best_scores_ind = scores[..., int(DIM/2):DIM].argmax(dim=-1, keepdim=True) + int(DIM/2)
    best_path = torch.zeros_like(inputs)
    # This handling of best_path_inds doesn't quite work as intended with 3+ dim input
    best_path_inds = path_inds[...,best_scores_ind].squeeze(-1).transpose(-2, -1)
    best_path.scatter_(dim=-1, index=best_path_inds, value=1.0)
    return best_path

# We initialize a random tensor
x = torch.rand([128, DIM]).to(**kwargs)
print(x)

best_path(torch.rand(10, DIM))

tensor([[0.5094, 0.6056, 0.0259,  ..., 0.3797, 0.8463, 0.0484],
        [0.3547, 0.9234, 0.8487,  ..., 0.1251, 0.8251, 0.8643],
        [0.4514, 0.4997, 0.9248,  ..., 0.0568, 0.8061, 0.1803],
        ...,
        [0.5334, 0.8792, 0.5630,  ..., 0.7294, 0.7334, 0.8180],
        [0.0151, 0.3987, 0.3550,  ..., 0.4042, 0.2250, 0.9816],
        [0.9972, 0.7213, 0.3172,  ..., 0.3397, 0.4160, 0.4699]],
       dtype=torch.float64)


tensor([[1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [16]:
# Turn its grad on, since we will change this tensor to minimize our loss
x.requires_grad = True
y_true = torch.Tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

print("Initially, the values in our tensor do not result in the desired argsort")
print(best_path(x))
print(y_true)

presampled_noises = get_presampled_noises(x.shape, num_samples=1000, noise="normal")
torch.norm(
    fixed_noise_perturbed(
        best_path, 
        presampled_noises=presampled_noises, 
        num_samples=1000, 
        sigma=0.25, 
        device=device
    )(x) - y_true, 
    dim=-1
)

# FenchelYoungLoss(ranks, device=device)(y_true, x)

# Initialize an SGD optimizer and do 200 steps
optim = torch.optim.SGD([x], 1.0)
# optim = torch.optim.LBFGS([x], .1)


Initially, the values in our tensor do not result in the desired argsort
tensor([[1., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 1.],
        [1., 1., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)
tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])


In [30]:
%%time 
BASE = 10

N_SAMPLES = 1000

for iteration in range(10*BASE+1):    
    presampled_noises = get_presampled_noises(x, num_samples=N_SAMPLES, noise="normal")
    def closure():
        optim.zero_grad()
    #     criterion = FenchelYoungLoss(ranks, sigma=0.25, device=device)
    #     loss = criterion(x, y_true).sum()
        perturbed_x = fixed_noise_perturbed(
            best_path, 
            presampled_noises=presampled_noises, 
            num_samples=N_SAMPLES, 
            sigma=0.25, 
            device=device
        )(x)
        loss = torch.norm(perturbed_x - y_true, dim=-1).mean()
        loss.backward()
        return loss
    optim.step(closure)
    if iteration % BASE== 0:
        perturbed_x = fixed_noise_perturbed(
            best_path, 
            presampled_noises=presampled_noises, 
            num_samples=N_SAMPLES, 
            sigma=0.25, 
            device=device
        )(x)
        loss = torch.norm(perturbed_x - y_true, dim=-1).mean()
#         print(perturbed_x)
        print(torch.abs(best_path(x) - y_true).sum().item())
        print(loss.item())
        
print("SGD has run???")

10.0
0.04560862041840651
10.0
0.0452581408411086
10.0
0.045151016317713635
10.0
0.04529177227512354
10.0
0.04501454448339936
10.0
0.0448788276075417
10.0
0.044686285659929596
10.0
0.04488033537437286
10.0
0.04476471284426817
10.0
0.044909720494043875
10.0
0.044411909550232165
SGD has run???
Wall time: 9.28 s


## CVXPyLayers - DAG

In [5]:
from cvxpylayers.torch import CvxpyLayer
import torch
import cvxpy as cp

In [6]:
# Define convex optimization model
num_n = 5
num_e = 5
e_out = [
    [0, 1],
    [2, 3],
    [4],
    [],
    [],
]
e_in = [
    [],
    [0],
    [1],
    [2, 4],
    [3],
]

n = cp.Variable(num_n)
e = cp.Variable(num_e)
s_in = cp.Parameter(1)
s_out = cp.Variable(2)
e_hat = cp.Parameter(num_e)

objective = cp.norm2(e-e_hat)
bound_constraints = [e >= 0, e <= 1]
flow_constraints = [
    s_in[0] - e[0] - e[1] == 0,
    e[0] - e[2] - e[3] == 0,
    e[1] - e[4] == 0,
    e[2] + e[4] + s_out[0] == 0,
    e[3] + s_out[1] == 0,
    
]
source_sink_constraints = [
    s_in[0] + s_out[0] + s_out[1] == 0,
]
constraints = bound_constraints + flow_constraints + source_sink_constraints

prob = cp.Problem(objective=cp.Minimize(objective), constraints=constraints)
dag_proj_layer = CvxpyLayer(problem=prob, parameters=[e_hat, s_in], variables=[e, s_out])

In [7]:
# layer(torch.Tensor([1.0]), torch.Tensor([1.0, 0.5]))
path_true = torch.Tensor([1.0, 0.0, 0.0, 1.0, 0.0])
e_arg = torch.Tensor([1.5, 1.0, 1.0, 0.0, 1.0])
e_arg.requires_grad = True
s_in_arg = torch.tensor([1.0])
e_res, s_res = dag_proj_layer(e_arg, s_in_arg)
loss = torch.norm(e_res - path_true)
loss.backward()
e_res

tensor([ 6.2500e-01,  3.7500e-01,  6.2500e-01, -1.2694e-10,  3.7500e-01],
       grad_fn=<_CvxpyLayerFnFnBackward>)

In [8]:
dag_proj_layer(torch.Tensor([e_arg.tolist(), e_arg.tolist()]), torch.Tensor([[1.0], [2.0]]))

(tensor([[ 6.2500e-01,  3.7500e-01,  6.2500e-01, -1.2694e-10,  3.7500e-01],
         [ 1.0000e+00,  1.0000e+00,  9.9997e-01,  3.6081e-05,  1.0000e+00]]),
 tensor([[-1.0000e+00,  1.6819e-10],
         [-2.0000e+00, -3.3548e-05]]))

In [9]:
dtype = torch.double
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

kwargs = {"dtype": dtype, "device": device}

DIM = 5

# We initialize a random tensor
x = torch.rand([128, DIM]).to(**kwargs)
print(x.shape)
source = torch.ones(x.shape[0], 1).to(**kwargs)
print(source.shape)

torch.Size([128, 5])
torch.Size([128, 1])


In [10]:
# Turn its grad on, since we will change this tensor to minimize our loss
x.requires_grad = True
y_true = torch.Tensor([1.0, 0.0, 0.0, 1.0, 0.0]).to(**kwargs)

print(y_true)

# Initialize an SGD optimizer and do 200 steps
optim = torch.optim.Adam([x], 1.0)
# optim = torch.optim.LBFGS([x], lr=1.0)


tensor([1., 0., 0., 1., 0.], dtype=torch.float64)


In [13]:
%%time 
import time
BASE = 1

fwd = 0
bwd = 0
for iteration in range(10*BASE+1):
    
    def closure():
        optim.zero_grad()
        start = time.time()
        dag_proj, _ = dag_proj_layer(x, source)
        end = time.time()
#         fwd += end - start
        loss = torch.norm(dag_proj - y_true, dim=-1).mean()
    #     + torch.norm(dag_proj - x)
    #     + torch.maximum(
    #         torch.norm(x, dim=-1) - torch.Tensor([6]).to(**kwargs), torch.Tensor([0]).to(**kwargs)
    #     ).mean()
        start = time.time()
        loss.backward()
        end = time.time()
#         bwd += end - start  
        return loss
    optim.step(closure)
    if iteration % BASE== 0:
        dag_proj, _ = dag_proj_layer(x, source)
        loss = torch.norm(dag_proj - y_true, dim=-1).mean()
#         print(perturbed_x)
#         print(dag_proj[:10,:7])
        print(torch.abs(dag_proj - y_true).sum().item())
        print(loss.item(), torch.sum(x, dim=-1).mean().item())
        
print("SGD has run???")
print("TIMES: ", fwd, bwd)

16.000512770950756
0.08191826200046354 1.6599763749581091
16.002033550906674
0.0819249876632022 1.0906809954782428
16.00333259463037
0.08193021047826303 0.6494827182949718
16.001760518178287
0.08192345153914424 0.2888068104523683
16.00227371371424
0.08192531921804438 -0.015194829761478186
16.002672576694863
0.08192671455432253 -0.2770450974642895
16.002848632752467
0.0819273362390188 -0.5055346821213603
16.003573304081602
0.0819306215730234 -0.7068369512523363
16.003164003461393
0.08192879223237422 -0.8855295932813281
16.00210037377061
0.08192460644265018 -1.0452757951581344
16.001309180195705
0.08192152458448611 -1.1883273407460293
SGD has run???
TIMES:  0 0
Wall time: 10.8 s


In [14]:
torch.round(dag_proj).sum(dim=0)

tensor([127.,   1.,   6., 121.,   1.], dtype=torch.float64,
       grad_fn=<SumBackward1>)

## CVXPyLayers - Tree

In [15]:
dtype = torch.double
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

kwargs = {"dtype": dtype, "device": device}

In [16]:
# Define convex optimization model
num_e = 14
num_sinks = 8
e = cp.Variable(num_e)
s_in = cp.Parameter(1)
s_out = cp.Variable(num_sinks)
e_hat = cp.Parameter(num_e)

objective = cp.norm2(e-e_hat)
bound_constraints = [e >= 0, e <= 1]
flow_constraints = [
    e[0] - e[2] - e[3] == 0,
    e[1] - e[4] - e[5] == 0,
    e[2] - e[6] - e[7] == 0,
    e[3] - e[8] - e[9] == 0,
    e[4] - e[10] - e[11] == 0,
    e[5] - e[12] - e[13] == 0,
] + [
    e[6+i] + s_out[i] == 0
    for i in range(num_sinks)
]
source_sink_constraints = [
    s_in[0] + cp.sum(s_out) == 0,
]
constraints = bound_constraints + flow_constraints + source_sink_constraints

prob = cp.Problem(objective=cp.Minimize(objective), constraints=constraints)
dag_proj_layer = CvxpyLayer(problem=prob, parameters=[e_hat, s_in], variables=[e, s_out])

In [17]:
DIM = 14

# We initialize a random tensor
x = torch.rand([128, DIM]).to(**kwargs)
print(x.shape)
source = torch.ones(x.shape[0], 1).to(**kwargs)
print(source.shape)

torch.Size([128, 14])
torch.Size([128, 1])


In [18]:
# Turn its grad on, since we will change this tensor to minimize our loss
x.requires_grad = True
y_true = torch.Tensor([1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

# print(best_path(x))
print(y_true)

dag_proj, _ = dag_proj_layer(x, source) 
print(torch.norm(dag_proj - y_true, dim=-1).mean())

# Initialize an SGD optimizer and do 200 steps
# optim = torch.optim.SGD([x], lr=10.0)
optim = torch.optim.LBFGS([x], lr=1.0)

tensor([1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor(1.5366, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [19]:
import time

In [20]:
%%time 
BASE = 1

fwd = 0
bwd = 0
for iteration in range(10*BASE+1):
    
    def closure():
        optim.zero_grad()
        start = time.time()
        dag_proj, _ = dag_proj_layer(x, source)
        end = time.time()
#         fwd += end - start
        loss = torch.norm(dag_proj - y_true, dim=-1).mean()
    #     + torch.norm(dag_proj - x)
    #     + torch.maximum(
    #         torch.norm(x, dim=-1) - torch.Tensor([6]).to(**kwargs), torch.Tensor([0]).to(**kwargs)
    #     ).mean()
        start = time.time()
        loss.backward()
        end = time.time()
#         bwd += end - start  
        return loss
    optim.step(closure)
    if iteration % BASE== 0:
        dag_proj, _ = dag_proj_layer(x, source)
        loss = torch.norm(dag_proj - y_true, dim=-1).mean()
#         print(perturbed_x)
#         print(dag_proj[:10,:7])
        print(torch.abs(dag_proj - y_true).sum().item())
        print(loss.item(), torch.sum(x, dim=-1).mean().item())
        
print("SGD has run???")
print("TIMES: ", fwd, bwd)

88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
88.00155916694962
0.4408341696149421 7.011729437540335
SGD has run???
TIMES:  0 0
Wall time: 16.4 s


In [21]:
x[:,6]

tensor([3.3032e+01, 1.2272e-03, 3.5869e+01, 2.7268e+01, 4.7818e-01, 3.0829e+01,
        6.7247e-01, 2.5717e+01, 3.3426e-01, 1.5892e-02, 1.4308e-01, 1.1910e-01,
        3.0708e-01, 4.4536e-02, 3.3555e-01, 7.5614e-01, 2.2788e+01, 2.5765e+01,
        2.7447e+01, 3.1402e+01, 2.7189e+01, 3.0408e+01, 2.7166e+01, 8.9052e-02,
        2.5729e+01, 2.2520e-01, 1.8217e-02, 2.5697e+01, 2.9008e+01, 1.7795e-01,
        5.9570e-01, 2.7840e+01, 2.9448e+01, 2.6707e+01, 2.4076e+01, 2.7410e+01,
        2.3254e+01, 2.2695e+01, 2.4997e-03, 2.1016e-02, 2.6753e+01, 2.5013e+01,
        2.7842e+01, 2.8003e+01, 2.4093e+01, 1.7035e-01, 2.7923e+01, 2.9427e+01,
        2.7241e+01, 2.7868e+01, 3.0894e+01, 2.5651e-01, 8.0247e-02, 2.6379e-01,
        2.6699e+01, 6.2519e-01, 3.7394e-01, 3.2489e+01, 2.5667e+01, 2.5440e+01,
        2.7112e+01, 2.5906e+01, 2.4713e+01, 2.7892e+01, 2.5580e+01, 2.6652e+01,
        3.1305e+01, 3.4211e+01, 2.6799e+01, 2.6907e+01, 2.6111e+01, 3.5460e+01,
        2.5712e+01, 2.0993e+01, 2.6971e+

In [22]:
dag_proj[:,[0,2,6]]

tensor([[ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00, -1.0554e-11],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00, -5.6148e-10],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 9.9999e-01, -9.9960e-07, -1.3740e-06],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00, -5.3321e-09, -1.3239e-08],
        [ 1.0000e+00,  1.0000e+00, -9.5956e-11],
        [ 1.0000e+00,  1.0000e+00,  4.2307e-10],
        [ 1.0000e+00,  1.0000e+00, -4.5745e-10],
        [ 1.0000e+00,  1.0000e+00,  7.7381e-11],
        [ 1.0000e+00,  1.0000e+00,  2.1403e-11],
        [ 1.0000e+00,  1.0000e+00,  6.7369e-11],
        [ 1.0000e+00,  1.0000e+00,  1.5159e-07],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00